# 1. Jednotkové testy


## 1.1 Proč jednotkové testy

- Jednotkové testy automaticky ověřují malé, izolované části kódu.
- Pomáhají odhalit regresi po změnách a dávají rychlou zpětnou vazbu.

V Pythonu řadu chyb odhalíme až při běhu programu. Proto je průběžné testování důležitá součást vývoje.


## 1.2 Použití v praxi: `unittest` a `pytest`


Ve standardní knihovně je modul `unittest` se všemi základními nástroji pro psaní testů. Testovací třídy obvykle dědí z `unittest.TestCase`.


**Příklad:** Máme třídu `Auto`, která na první pohled vypadá dobře. Ověřme to testy.


In [ ]:
%%file auto.py
# -*- coding: utf8 -*-
class Auto(object):
    def __init__(self, spotreba, rychlost):
        self.spotreba = spotreba
        self.rychlost = rychlost
        self.cas = 0
        self.vzdalenost = 0
        self.nadrz = 50
        
    def ujed(self, vzdalenost):
        self.vzdalenost += vzdalenost
        self.cas += vzdalenost / self.rychlost
        self.nadrz -= vzdalenost * self.spotreba



Napíšeme několik testů a podíváme se, kde implementace selhává.


In [ ]:
%%file auto_test.py

# -*- coding: utf8 -*-
from auto import Auto
import unittest

class AutoTest(unittest.TestCase):             # Dědíme z třídy unittest.TestCase
    def test_vypocet_spotreby(self):
        auto = Auto(10, 200)                   # Dost žere, ale je rychlé
        nadrz1 = auto.nadrz
        auto.ujed(100)
        nadrz2 = auto.nadrz
        self.assertEqual(10, nadrz1 - nadrz2)  # Víme, že auto mělo spotřebovat 10 litrů
        
    def test_neprazdna_nadrz(self):
        auto = Auto(8, 100)
        with self.assertRaises(Exception):
            auto.ujed(1000)                    # Dojde benzín
        self.assertTrue(auto.nadrz == 0)       # I poté musí nádrž být nejhůře prázdná    
        
    def test_nesmyslnych_aut(self):
        with self.assertRaises(Exception):
            auto = Auto(0, 100)                # Auto bez spotřeby!
        with self.assertRaises(Exception):
            auto = Auto(10, 0)                 # Auto, které neumí jezdit!
        with self.assertRaises(Exception):
            auto = Auto(-10, 100)              # Auto, které vyrábí benzín.

    def test_nezaporna_vzdalenost(self):       # Metody začínající na "test_" jsou automaticky spuštěny
        auto = Auto(8, 100)
        with self.assertRaises(Exception):
            auto.ujed(-1)
    
if __name__ == "__main__":
    unittest.main()                 # Tímto pustíme testy

In [ ]:
!python auto_test.py

Ukázalo se několik problémů:

- neřešíme jednotky spotřeby,
- lze vytvořit nesmyslné auto,
- je možné "přečerpat" nádrž,
- záporná vzdálenost dává nesmyslný výsledek.

Implementaci upravíme a testy pustíme znovu.


In [ ]:
%%file auto.py

# -*- coding: utf8 -*-
from __future__ import division

class Auto(object):
    def __init__(self, spotreba, rychlost):
        if spotreba <= 0:
            raise Exception("Auto musí mít kladnou spotřebu.")
        if rychlost <= 0:
            raise Exception("Auto musí jezdit kladnou rychlostí.")
        self.spotreba = spotreba
        self.rychlost = rychlost
        self.cas = 0
        self.vzdalenost = 0
        self.nadrz = 50

    def ujed(self, vzdalenost):
        if vzdalenost < 0:
            raise Exception("Vzdálenost musí být nezáporná.")
        if (vzdalenost * self.spotreba / 100) > self.nadrz:
            # Auto ujede, kolik může, a pak vyhodí výjimku.
            skutecna_vzdalenost = 100 * (self.nadrz / self.spotreba)
            self.ujed(skutecna_vzdalenost)  # Rekurze
            raise Exception("Došel benzín")
        self.vzdalenost += vzdalenost
        self.cas += vzdalenost / self.rychlost
        self.nadrz -= (vzdalenost * self.spotreba / 100)


In [ ]:
!python auto_test.py

Po úpravě implementace testy procházejí.


### 1.2.1 `pytest`

`pytest` je v praxi velmi rozšířený framework. Oproti čistému `unittest` obvykle zjednodušuje zápis testů i jejich spouštění.


In [ ]:
%%file test_auto_pytest.py

# -*- coding: utf8 -*-
from auto import Auto
import pytest

def test_vypocet_spotreby():
    auto = Auto(10, 200)                   # Dost žere, ale je rychlé
    nadrz1 = auto.nadrz
    auto.ujed(100)
    nadrz2 = auto.nadrz
    assert 10 == nadrz1 - nadrz2           # Víme, že auto mělo spotřebovat 10 litrů

def test_neprazdna_nadrz():
    auto = Auto(8, 100)
    with pytest.raises(Exception):
        auto.ujed(1000)                    # Dojde benzín
    assert auto.nadrz == 0                 # I poté musí nádrž být nejhůře prázdná    

def test_nesmyslnych_aut():
    with pytest.raises(Exception):
        auto = Auto(0, 100)                # Auto bez spotřeby!
    with pytest.raises(Exception):
        auto = Auto(10, 0)                 # Auto, které neumí jezdit!
    with pytest.raises(Exception):
        auto = Auto(-10, 100)              # Auto, které vyrábí benzín.

def test_nezaporna_vzdalenost():       # Metody začínající na "test_" jsou automaticky spuštěny
    auto = Auto(8, 100)
    with pytest.raises(Exception):
        auto.ujed(-1)

Testy ze souboru můžeme spustit například takto:


In [ ]:
!pytest -vv test_auto_pytest.py

Ještě jednodušší je nechat `pytest` projít aktuální adresář a spustit všechny nalezené testy.


In [ ]:
!pytest -vv .

## 1.3 Testování ve VSCode

VSCode umí testy automaticky najít a spouštět přímo z GUI. Dostupné jsou i přehledné výsledky běhu testů.

K funkcím testování se dostanete přes ikonu **Testing** (zkumavka) v levém panelu.


## 1.4 Mypy

Pokud používáme type hinty, můžeme kontrolovat jejich konzistenci nástrojem `mypy`.


In [ ]:
!mypy mujbalik/mujbalik/

Pokud máte dobře nastavenou statickou kontrolu v IDE, část těchto problémů uvidíte už při psaní kódu.
